In [2]:
# define dependency 
from tensorflow.keras.models import load_model

# load model 
model = load_model('sent_clf_model_cc_data.keras')

# check model info 
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       426,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 31)             │         2,015 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 873,280 (3.33 MB)

 Trainable params: 436,639 (1.67 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 436,641 (1.67 MB)

In [7]:
import pickle
import nltk
import numpy as np
import pandas as pd
import random
from nltk.stem import WordNetLemmatizer

In [5]:
with open('words_in_all_patterns.pkl', 'rb') as file:
    words_in_all_patterns = pickle.load(file)

words_in_all_patterns[:5], len(words_in_all_patterns)

(['abandoned', 'abandonment', 'ability', 'able', 'abortion'], 3330)

In [8]:
with open('all_tags.pkl', 'rb') as f:
    all_tags = pickle.load(f)

all_tags[:5], len(all_tags)

(['depression',
  'anxiety',
  'parenting',
  'self-esteem',
  'relationship-dissolution'],
 31)

In [6]:
lemmatizer = WordNetLemmatizer()

In [16]:
def preprocess_user_message(user_message):
    # Tokenizing and lemmatizing user message
    words_in_user_message = nltk.word_tokenize(user_message)
    words_in_user_message = [lemmatizer.lemmatize(word.lower()) for word in words_in_user_message]

    print("words_in_user_message:", words_in_user_message)

    # Converting user message to vector
    user_message_vector = [0] * len(words_in_all_patterns)
    
    # w: word in user input
    # word: word in the list of all words
    for w in words_in_user_message:
        for i, word in enumerate(words_in_all_patterns):
            if word == w:
                user_message_vector[i] = 1
    
    user_message_vector = np.array(user_message_vector)
    print("User message vector:", user_message_vector)
    
    return user_message_vector

In [26]:
def predict_tag_for_user_message(user_message_vector):
    res = model.predict(np.array([user_message_vector]), verbose = 0)[0]
    ERROR_THRESHOLD = 0.4

    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key = lambda x: x[1], reverse = True)

    # returns all tags whose prob. > 0.25
    tag_and_probability = []
    for r in results:
        tag_and_probability.append({'intent': all_tags[r[0]], 'probability': str(r[1])})

    print("tag_and_probability:", tag_and_probability)

    if len(tag_and_probability) != 0:
        return tag_and_probability
    else:
        return [{"intent": "unknown"}]

In [27]:
predict_tag_for_user_message(preprocess_user_message("How can I manage anger?"))

words_in_user_message: ['how', 'can', 'i', 'manage', 'anger', '?']
User message vector: [0 0 0 ... 0 0 0]
tag_and_probability: [{'intent': 'anger-management', 'probability': '0.9999893'}]


[{'intent': 'anger-management', 'probability': '0.9999893'}]